<a href="https://colab.research.google.com/github/kingtuler1454/torch/blob/main/5kab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   Необходимо загрузить исходный набор данных и соответствующие метки классов.
2.   Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.
3.   Написать модель нейронной сети для решения задачи классификации.
4.   Описать пайплайн предобработки данных. **ВАЖНО**: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).
4.   Написать `train loop` (цикл обучения). Провести эксперименты по обучению с различными значениями параметров `learning rate` (скорость обучения) и `batch size` (размер мини-пакета). Выбрать по 3 значения для `learning rate` и `batch size` (итоговое количество экспериментов будет 9).
5.   Для каждого проведенного эксперимента вывести графики для значения функции потерь (ось `x` - итерация обучения/номер эпохи; ось `y` - значение функции потерь) и выбранной метрики качества (ось `x` - итерация обучения/номер эпохи; ось `y` - значение метрики качества). Графики необходимо выводить как для обучающей, так и для валидационной выборки.
6.   Оценить качество работы модели на тестовой выборке.
7.   Сделайте выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать?
8.   Сохранить обученную модель.
9.   Выполните повторную инициализацию модели и загрузку весов.  Продемонстрируйте работоспособность модели (пропустите через нее какой-то отзыв/рецензию и выведите результат).


https://www.kaggle.com/code/shivammehta007/spam-not-spam-classifier-with-pytorch

In [2]:
import pandas as pd
from numpy import loadtxt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Mon Dec 19 17:22:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    26W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Необходимо загрузить исходный набор данных и соответствующие метки классов.

In [5]:
    
with open("/content/number_star.txt","r+") as file:
  number_star=file.read()
  number_star=number_star.split(", ")

with open("/content/text_opinions.txt","r+") as file:
  text_opinion=file.read()
  text_opinion=text_opinion
  text_opinion=text_opinion.split("', '")

Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.

In [6]:
train_opinion=text_opinion[0:240]
train_star=number_star[0:240]
train_opinion=train_opinion+text_opinion[1840:2080]
train_star=train_star+number_star[1840:2080]
train_opinion=train_opinion+text_opinion[2380:2620]
train_star=train_star+number_star[2380:2620]
train_opinion=train_opinion+text_opinion[2800:3040]
train_star=train_star+number_star[2800:3040]
train_opinion=train_opinion+text_opinion[3134:3374]
train_star=train_star+number_star[3134:3374]


test_opinion=text_opinion[240:270]
test_star=number_star[240:270]
test_opinion=test_opinion+text_opinion[2080:2110]
test_star=test_star+number_star[2080:2110]
test_opinion=test_opinion+text_opinion[2620:2650]
test_star=test_star+number_star[2620:2650]
test_opinion=test_opinion+text_opinion[3040:3070]
test_star=test_star+number_star[3040:3070]
test_opinion=test_opinion+text_opinion[3374:3370]
test_star=test_star+number_star[3374:3370]


valid_opinion=text_opinion[270:300]
valid_star=number_star[270:300]
valid_opinion+=text_opinion[2110:2140]
valid_star+=number_star[2650:2680]
valid_opinion+=text_opinion[2650:2680]
valid_star+=number_star[3040:3070]
valid_opinion+=text_opinion[3370:3400]
valid_star+=number_star[3370:3400]

1200


Написать модель нейронной сети для решения задачи классификации.


In [7]:
train_df = pd.DataFrame({"star": train_star, "text": train_opinion})

remove_non_alphabets =lambda x: re.sub(r'[^а-яА-Я]',' ',x)
tokenize = lambda x: word_tokenize(x)
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]
lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

In [ ]:
train_df['text'] = train_df['text'].apply(remove_non_alphabets)
train_df['text'] = train_df['text'].apply(tokenize) # [ word_tokenize(row) for row in data['email']]
train_df['text'] = train_df['text'].apply(stem)
train_df['text'] = train_df['text'].apply(leammtizer)
train_df['text'] = train_df['text'].apply(lambda x: ' '.join(x))

In [11]:
max_words = 10000
cv = CountVectorizer(max_features=max_words, stop_words='english')
sparse_matrix = cv.fit_transform(train_df['text'] ).toarray()
sparse_matrix.shape
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(train_df['text']))

In [13]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 5)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

model = LogisticRegression()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).float()

TypeError: ignored